In [30]:
question = "select average forecast prices for corn for Jan 2024"

In [1]:
%pip install sqlalchemy pymysql torch transformers bitsandbytes accelerate sqlparse openai langchain_community


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:

# dbutils.library.restartPython()

In [3]:
hostname = "ingmysql001.mysql.database.azure.com"
dbname = "price_forecasting_dev"
jdbcPort = 3306
username = "sqladmin"
password = "Ing@098123"

In [4]:
import pymysql
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sqlparse
from openai import OpenAI
import textwrap

In [5]:
!iwconfig

/bin/bash: line 1: iwconfig: command not found


In [6]:
conn = pymysql.connect(
        host=hostname,
        user=username,
        password = password,
        db=dbname,
        )

def read_sql(query):
    try:
        return pd.read_sql(query, conn)
    except Exception as e:
        return pd.DataFrame()

def read_table(table):
    try:
        return pd.read_table(table, conn)
    except Exception as e:
        # print(e)
        return pd.DataFrame()


In [7]:
# Python Program to Get IP Address
import socket
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)

print("Your Computer Name is:" + hostname)
print("Your Computer IP Address is:" + IPAddr)

Your Computer Name is:a15d03153a57
Your Computer IP Address is:172.28.0.12


In [8]:
# read_sql('''SELECT *
# FROM forecast_price_master_product''')

In [9]:
# Check if GPU is available?
available_memory = torch.cuda.get_device_properties(0).total_memory
print(torch.cuda.is_available(), available_memory)

True 15835660288


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [10]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 16e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [31]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Use the Example mentioned to learn for question and it's answer
- The data contains commodites as follows so in case user input is in any other case use the case as per these records 'RBOB Gasoline', 'Crude Oil', 'LNG', 'Natgas', 'Heating Oil', 'Brent Crude', 'Ethanol','Palladium', 'Platinum', 'Silver', 'Gold', 'Copper', 'Aluminium','Cotton', 'Rice', 'Kansas Wheat', 'NZ Wmp', 'Soybeans', 'Soybean Meal', 'Arabica Coffee', 'NFDM', 'Cbot Wheat', 'Robusta Coffee', 'Soybean Oil', 'Palm Oil', 'Coffee', 'Cocoa', 'Corn', 'Sugar'
- The data contains models Multivariate, UNIVARIATE, Simple Model
- The data contains snapshotdate when the forecast was created
- The data contains horizon/outlook period in future months 1M to 12M
- Schema contains the foreign and primary keys use these key to join the tables
- Don't use columns which are not present in the create table schemas
### Database Schema
This query will run on a database whose schema is represented in this string:

create table categories_master
(
    category_id   int auto_increment
        primary key,
    category_name varchar(20)                           null,
    created_at    datetime    default CURRENT_TIMESTAMP null,
    updated_at    datetime    default CURRENT_TIMESTAMP null on update CURRENT_TIMESTAMP,
    updated_by    varchar(75) default (current_user())  null
)
    charset = utf8mb3;

create table dates_master
(
    date_id        int                                   not null
        primary key,
    date           date                                  not null,
    year           int                                   not null,
    quarter        char(2)                               not null,
    month          int                                   not null,
    day            int                                   not null,
    day_of_week    varchar(10)                           not null,
    month_name     varchar(10)                           not null,
    quarter_name   char(2)                               not null,
    week_of_year   int                                   not null,
    day_of_year    int                                   not null,
    fiscal_year    int                                   not null,
    fiscal_quarter char(2)                               not null,
    is_holiday     tinyint(1)  default 0                 not null,
    is_weekend     bit         default b'0'              not null,
    created_at     datetime    default CURRENT_TIMESTAMP null,
    updated_at     datetime    default CURRENT_TIMESTAMP null on update CURRENT_TIMESTAMP,
    updated_by     varchar(75) default (current_user())  null
)
    charset = utf8mb3;

create table exchanges_master
(
    exchange_id   int auto_increment
        primary key,
    exchange_name varchar(50)                           null,
    created_at    datetime    default CURRENT_TIMESTAMP null,
    updated_at    datetime    default CURRENT_TIMESTAMP null on update CURRENT_TIMESTAMP,
    updated_by    varchar(75) default (current_user())  null
)
    charset = utf8mb3;

create table geo_regions_master
(
    geo_region_id int auto_increment
        primary key,
    region        varchar(100)                          null,
    country       varchar(100)                          null,
    created_at    datetime    default CURRENT_TIMESTAMP null,
    updated_at    datetime    default CURRENT_TIMESTAMP null on update CURRENT_TIMESTAMP,
    updated_by    varchar(75) default (current_user())  null
)
    charset = utf8mb3;

create table models_master
(
    model_id   int auto_increment
        primary key,
    model_name varchar(20)                           null,
    created_at datetime    default CURRENT_TIMESTAMP null,
    updated_at datetime    default CURRENT_TIMESTAMP null,
    updated_by varchar(75) default (current_user())  null
)
    charset = utf8mb3;

create table source_types_master
(
    source_id   int auto_increment
        primary key,
    source_name varchar(20)                           not null,
    created_at  datetime    default CURRENT_TIMESTAMP null,
    updated_at  datetime    default CURRENT_TIMESTAMP null on update CURRENT_TIMESTAMP,
    updated_by  varchar(75) default (current_user())  null
)
    charset = utf8mb3;

create table commodities_master
(
    commodity_id   int auto_increment
        primary key,
    source_id      int                                   not null,
    country_id     int                                   not null,
    category_id    int                                   not null,
    actual_name    varchar(100)                          not null,
    commodity_name varchar(100)                          not null,
    display_name   varchar(100)                          not null,
    exchange_id    int                                   not null,
    website        varchar(200)                          not null,
    created_at     datetime    default CURRENT_TIMESTAMP null,
    updated_at     datetime    default CURRENT_TIMESTAMP null on update CURRENT_TIMESTAMP,
    updated_by     varchar(75) default (current_user())  null,

)
    charset = utf8mb3;


create table com_var_mappings_master
(
    my_row_id    bigint unsigned                      not null invisible primary key,
    commodity_id int                                  null,
    variable_id  int                                  null,
    created_at   datetime    default (now())          null,
    updated_at   datetime    default (now())          null,
    updated_by   varchar(75) default (current_user()) null,

);


create table forecast_price_master_product
(
    my_row_id              bigint unsigned                       not null invisible primary key,
    commodity_id           int                                   not null,
    model_id               int                                   not null,
    predicted_for_month_id int                                   not null,
    snapshot_month_id      int                                   not null,
    outlook_duration       char(3)                               null,
    model_output           float                                 null,
    actual_value           float                                 null,
    ape_value              float                                 null,
    lower_bound_95         float                                 null,
    upper_bound_95         float                                 null,
    lower_bound_65         float                                 null,
    upper_bound_65         float                                 null,
    change_percentage      float                                 null,
    base_value             float                                 null,
    category_id            int                                   not null,
    updated_at             datetime    default CURRENT_TIMESTAMP null,
    updated_by             varchar(75) default (current_user())  null,


);

create table input_variable_correlation_product
(
    my_row_id         bigint unsigned                       not null invisible primary key,
    commodity_id      int                                   not null,
    source_id         int                                   not null,
    target_id         int                                   not null,
    snapshot_month_id int                                   not null,
    value             float                                 not null,
    updated_at        datetime    default CURRENT_TIMESTAMP null,
    updated_by        varchar(75) default (current_user())  null,

)
    charset = utf8mb3;

create table multivariate_variable_importance_product
(
    commodity_id        int                                   not null,
    variable_id         int                                   not null,
    variable_rank       float                                 null,
    variable_importance float                                 null,
    variable_weight     float                                 null,
    snapshot_month_id   int                                   not null,
    updated_at          datetime    default CURRENT_TIMESTAMP null,
    updated_by          varchar(75) default (current_user())  null,
    primary key (commodity_id, variable_id, snapshot_month_id),



)
    charset = utf8mb3;

create table price_history_raw_product
(
    my_row_id     bigint unsigned                       not null invisible primary key,
    commodity_id  int                                   not null,
    trade_date_id int                                   not null,
    low_price     varchar(255) charset utf8mb3          null,
    high_price    varchar(255) charset utf8mb3          null,
    open_price    varchar(255) charset utf8mb3          null,
    close_price   float                                 null,
    updated_at    datetime    default CURRENT_TIMESTAMP null,
    updated_by    varchar(75) default (current_user())  null,

  );


create table sentiment_raw_data_product
(
    my_row_id          bigint unsigned                       not null invisible primary key,
    commodity_id       int                                   not null,
    publishing_date_id int                                   not null,
    title              varchar(150)                          null,
    url                varchar(300)                          null,
    updated_at         datetime    default CURRENT_TIMESTAMP null,
    updated_by         varchar(75) default (current_user())  null,
    )
    charset = utf8mb3;


create table sentiment_trend_data_product
(
    news_date_id    int                                   not null,
    commodity_id    int                                   not null,
    title           varchar(100)                          not null,
    url             varchar(300)                          not null,
    sentiment_tag   varchar(20)                           not null,
    sentiment_score float                                 null,
    updated_at      datetime    default CURRENT_TIMESTAMP null,
    updated_by      varchar(75) default (current_user())  null,
    primary key (news_date_id, commodity_id, title),

)
    charset = utf8mb3;

### Joins
-- Use the contraints mentioned below as primary and foreign keys to join the tables
-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id<|eot_id|><|start_header_id|>assistant<|end_header_id|>

foreign key (forecast_id) references adjusted_forecast_config_master (forecast_id)
foreign key (commodity_id) references commodities_master (commodity_id),
foreign key (variable_id) references commodities_master (commodity_id)
foreign key (commodity_id) references commodities_master (commodity_id),
foreign key (model_id) references models_master (model_id),
foreign key (predicted_for_month_id) references dates_master (date_id),
foreign key (snapshot_month_id) references dates_master (date_id),
foreign key (category_id) references categories_master (category_id)
foreign key (commodity_id) references commodities_master (commodity_id),
foreign key (source_id) references commodities_master (commodity_id),
foreign key (target_id) references commodities_master (commodity_id),
foreign key (snapshot_month_id) references dates_master (date_id)
foreign key (commodity_id) references commodities_master (commodity_id),
foreign key (snapshot_month_id) references dates_master (date_id)
foreign key (commodity_id) references commodities_master (commodity_id),
foreign key (trade_date_id) references dates_master (date_id)
foreign key (commodity_id) references commodities_master (commodity_id),
foreign key (publishing_date_id) references dates_master (date_id)
foreign key (news_date_id) references dates_master (date_id),
foreign key (commodity_id) references commodities_master (commodity_id)
### Example
Question: find all sentiments news for corn
Answer: Select *
from sentiment_trend_data_product inner join commodities_master on commodities_master.commodity_id = sentiment_trend_data_product.commodity_id where commodities_master.actual_name = 'Corn'

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [32]:
def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [44]:
generated_sql = generate_query(question)
data_df= read_sql(generated_sql)

In [37]:
prompt_template = """You are a Business and Data analyst assistant in converting questions asked in plain English language into mathematical, graphical and data driven answers with summary
Here are the details you'll need:

The data retrieved from the in tabular format
{data_df}

Instructions:
1. Return Summarised data
2. USe the websites livemint and Bloomberg to access recent sentiments to provide events and news for the commodity asked


Your Task:
Based on the given dataframe, English question, and urls on internet, present the summary and charts for a business user.
English Question: {question}"""

def create_prompt(prompt_template, data_df, question):
    prompt = prompt_template.format(data_df = data_df, question = question)
    return prompt

In [38]:

prompt_final = create_prompt(prompt_template, data_df, question)

In [35]:
client = OpenAI(api_key='sk-proj-GDMIMdcUqBSlmBRHxNbOT3BlbkFJZ3M5h9XRb4t4rqqC8eT1') # Pass the API key here
def generate_response(prompt_final):
    try:
      response = client.chat.completions.create(model="gpt-4o",messages=[{"role": "user", "content": prompt_final}
                                                                         ,{"role": "user", "content": 'Use the data from data_df input only'}])
      return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [42]:
generate_response(prompt_final)